In [3]:
import pysam
import pandas as pd
import numpy as np

In [8]:
def parse_attribute(attr):
    '''get gene_id from attribute
    '''
    for item in attr.split(';'):
        key, value = item.split('=')
        if key == 'ID':
            return value.replace('gene:', '')


def get_gene_is_reverse(gff_path):
    GFF_COLUMNS = ['seqid', 'source', 'type', 'start', 'end', 
                   'score', 'strand', 'phase', 'attribute']
    gff_df = pd.read_csv(gff_path, sep='\t', comment='#', names=GFF_COLUMNS)
    # keep type = 'gene'
    gff_df = gff_df[gff_df['type'].isin(['gene'])]
    # get gene_id
    gff_df['gene_id'] = gff_df['attribute'].map(parse_attribute)
    gff_df['is_reverse'] = gff_df['strand'].map(lambda x: False if x=='+' else True)
    
    return gff_df.set_index('gene_id')['is_reverse'].T.to_dict()

In [39]:
def load_polya_data(polya_data):
    # polya_data = '/public/home/mowp/test/fast5_api/fhh.polyacaller.ann.tsv'
    df = pd.read_csv(polya_data, sep='\t', index_col=0)
    df['read_is_reverse'] = df['read_type'].map(lambda x: True if x == 'polyT' else False)

    # a dict like this: {'read_id': True or False}
    read_is_reverse = df['read_is_reverse'].to_dict()
    # a dict like this: {'read_id': 'gene_id'}
    read_gene_id = df['gene_id'].to_dict()
    # a dict like this: {'read_id': 100.0}
    read_tail_length = df['tail_length'].to_dict()
    return read_is_reverse, read_gene_id, read_tail_length

In [52]:
gff_path = '/public/home/mowp/db/Arabidopsis_thaliana/gff3/Arabidopsis_thaliana.TAIR10.46.gff3'
gene_is_reverse_dict = get_gene_is_reverse(gff_path)

polya_data = '/public/home/mowp/test/fast5_api/fhh.polyacaller.ann.tsv'
read_is_reverse_dict, read_gene_id_dict, read_tail_length_dict = load_polya_data(polya_data)

In [104]:
infile = '/public/home/mowp/test/nanopore_cdna/aligned_data/fhh.adjust.mm2.sorted.bam'
outfile = '/public/home/mowp/test/nanopore_cdna/aligned_data/fhh.tagged.mm2.sorted.bam'
with pysam.AlignmentFile(infile, 'rb') as inbam, pysam.AlignmentFile(outfile, 'wb', template=inbam) as outbam:
    # add tags: pa for polya_length
    #           gi for gene_id
    for read in inbam.fetch():
        # only output mapped reads
        if not read.is_unmapped:
            read_is_reverse = read_is_reverse_dict[read.query_name]
            read_gene_id = read_gene_id_dict[read.query_name]
            read_tail_length = read_tail_length_dict[read.query_name]
            # 能被注释到gene_id的执行如下操作
            try:
                gene_is_reverse = gene_is_reverse_dict[read_gene_id]
                read.set_tag('gi', read_gene_id)
                if gene_is_reverse is read.is_reverse:
                    read.set_tag('pa', read_tail_length)
                else:
                    continue
            # 不能注释的直接输出，有可能是新的转录本
            except KeyError:
                read.set_tag('gi', 'None')
                read.set_tag('pa', read_tail_length)
            outbam.write(read)

In [103]:
read_tail_length

115.13635194237202

In [92]:
read.get_tags()

[('NM', 69),
 ('ms', 1499),
 ('AS', 1331),
 ('nn', 0),
 ('ts', '+'),
 ('tp', 'P'),
 ('cm', 420),
 ('s1', 1495),
 ('s2', 0),
 ('de', 0.02329999953508377),
 ('cs',
  ':23-ag:146+g:21+g:1*ca*ga:9+c:54*tc:2-cta:2-a:17~gt82ag:23-g:49*ag:53-g:54-a:6-t:88+c:3~gt209ag:30+aa:42+c:3*at:27*gt:16~gt100ag:133-ct:48-ta:183-tga:19~gt78ag:17*ga*ag:134~gt112ag:27-g:26-a:71-a:76*ga:1-a:5*ga:18+ag*tc:10-ttt:7-tt:62-aaagaagagaaaaaggggg:45+a:44-a:1*tg:34*ga:3'),
 ('rl', 84),
 ('pa', 115.1363525390625),
 ('ge', 'AT1G01010'),
 ('gi', nan)]

In [60]:
df

,read_type,tail_start,tail_end,samples_per_nt,tail_length,transcript_id,gene_id,read_is_reverse
read_id,,,,,,,,
84778c00-f131-4c7e-a741-a3d17a72ae95,polyT,1176,2572,9.293564,176.251014,NaN,NaN,True
39f88611-6c7f-439d-a1bc-4baff95c9eb8,polyA,6094,7480,9.681818,144.394366,NaN,NaN,False
4d6c098a-e098-423a-a315-6c45a3b51b28,polyA,2334,2706,9.648069,88.722420,NaN,NaN,False
558d297a-c033-4663-9696-525c96f2946e,polyT,2288,2924,10.660920,61.533154,NaN,NaN,True
314c0d3a-3bec-4989-bbb0-737841bd535c,polyT,830,1052,9.890995,24.062290,NaN,NaN,True
...,...,...,...,...,...,...,...,...
f56fe844-910b-4c46-8520-46afbe8739ea,polyA,13616,16026,9.446951,279.455253,NaN,NaN,False
f91087bc-6a2c-4212-8cc3-ccb285d55583,polyT,986,1116,10.048059,14.132082,NaN,NaN,True
9b91b8c2-ec34-496d-b96e-027fca06edd6,polyT,1512,1512,10.549751,2.274935,NaN,NaN,True


In [105]:
i

'84778c00-f131-4c7e-a741-a3d17a72ae95'